In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/multi/202206/0609/data/prep_data_dotsplits.csv')
df.columns = ['question', 'answer']
df = df[:30]
df.answer = df.answer.apply(lambda x: '\t ' + str(x) + ' \n')
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,question,answer
0,정말 어려운 상황에서 너무나도 힘들어서 질문을 드립니다. 부디 자세한 설명해주시면 ...,\t 차량의 경우 현재 점유자인 아버지를 피고로 하여 차량 인도 청구소송 및 차량 ...
1,1년 전에 대포 통장이로 인해 피해를 봐서 재판 결과가 불기소 유통 지을 받았거든요...,\t 소장 내용에 대한 검토가 필요하므로 소장을 지참하고 상담을 받으시기 바랍니다 \n
2,연고도 다르고 일면식도 없는 5살위의 형을 일을 통해 알게 됐습니다.제가 주식투자를...,\t 별도의 구체적 상담이 필요하나 실제 채무가 없음에도 공증을 가지고 채권자가 강...
3,1 21615 이행권고결정판결 2 211523 채권압류 및 추심명령 인용 21153...,\t 별도의 구체적 상담이 필요하나 이행 권고 결정은 기판력의 제한이 없으므로 소멸...
4,4천만 원 정도의 피해가 있고 고소를 했는데 인지사건으로 넘어가서 검찰에서 수사 중...,\t 가해자가 위 사건으로 구속 기소되었다면 가해자를 피고로 하여 민사소송 및 강제...


In [ ]:
# from __future__ import print_fuction
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

batch_size = 16
epochs = 25
latent_dim = 256
num_samples = 3



input_texts = []
target_texts = []


input_characters = set()
target_characters = set()
for idx in df.index :
    input_text = str(df.loc[idx, 'question'])
    target_text = '\t' + str(df.loc[idx, 'answer']) + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)



# for line in input_texts:
#     for char in str(line):
#         if str(char) not in input_characters:
#             input_characters.add(str(char))
# for line in target_texts:
#     for char in str(line):
#         if str(char) not in target_characters:
#             target_characters.add(str(char))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.

encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.2)

# Save modelmodel.save('s2s.h5') 
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states 

# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length) :
                stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

for seq_index in range(3):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)



Number of samples: 30
Number of unique input tokens: 609
Number of unique output tokens: 221
Max sequence length for inputs: 2705
Max sequence length for outputs: 118
Epoch 1/25
2/2 [==============================] - 5s 1s/step - loss: 0.1588 - val_loss: 0.1575
Epoch 2/25
2/2 [==============================] - 1s 378ms/step - loss: 0.1488 - val_loss: 0.1564
Epoch 3/25
2/2 [==============================] - 1s 370ms/step - loss: 0.1479 - val_loss: 0.1440
Epoch 4/25
2/2 [==============================] - 1s 362ms/step - loss: 0.1396 - val_loss: 0.1424
Epoch 5/25
2/2 [==============================] - 1s 363ms/step - loss: 0.1380 - val_loss: 0.1417
Epoch 6/25
2/2 [==============================] - 1s 376ms/step - loss: 0.1375 - val_loss: 0.1404
Epoch 7/25
2/2 [==============================] - 1s 361ms/step - loss: 0.1365 - val_loss: 0.1396
Epoch 8/25
2/2 [==============================] - 1s 373ms/step - loss: 0.1357 - val_loss: 0.1392
Epoch 9/25
2/2 [==============================] - 1s

In [3]:
cd /content/drive/MyDrive/multi/202206/0609

/content/drive/MyDrive/multi/202206/0609


In [4]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
from konlpy.tag import Okt, Mecab
mecab = Mecab()
print(mecab.pos(df.question[0]))
text_1  = []
text_pos = ''
for (val, nn) in mecab.pos(df.question[0]):
    text_1.append(val)
for (val, nn) in mecab.pos(df.question[0]):
    text_pos += val + ' '
text_1, text_pos

In [ ]:
mecab.pos(df.question[0][:20])

[('정말', 'MAG'),
 ('어려운', 'VA+ETM'),
 ('상황', 'NNG'),
 ('에서', 'JKB'),
 ('너무나', 'MAG'),
 ('도', 'JX'),
 ('힘들', 'VA'),
 ('어', 'EC')]

In [6]:
# from __future__ import print_fuction
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd
from konlpy.tag import Okt, Mecab
mecab = Mecab()

stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

batch_size = 16
epochs = 25
latent_dim = 256
num_samples = 3
# max_sentence_len = 10


input_texts = []
target_texts = []


input_characters = set()
target_characters = set()
for idx in df.index :

    input_text = [ val for (val, nn) in mecab.pos(df.loc[idx, 'question'][::-1]) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]

    ans_list = [ val for (val, nn) in mecab.pos(df.loc[idx, 'answer']) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]
    ans_list.insert(0, '\t')
    ans_list.insert(-1, '\n')
    target_text = ans_list
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.

encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.2)

# Save modelmodel.save('s2s.h5') 
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states 

# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length) :
                stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

for seq_index in range(3):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)



Number of samples: 30
Number of unique input tokens: 1338
Number of unique output tokens: 194
Max sequence length for inputs: 695
Max sequence length for outputs: 26
Epoch 1/25
2/2 [==============================] - 20s 7s/step - loss: 0.1297 - val_loss: 0.1446
Epoch 2/25
2/2 [==============================] - 13s 6s/step - loss: 0.1284 - val_loss: 0.1400
Epoch 3/25
2/2 [==============================] - 13s 6s/step - loss: 0.1235 - val_loss: 0.1343
Epoch 4/25
2/2 [==============================] - 13s 6s/step - loss: 0.1199 - val_loss: 0.1389
Epoch 5/25
2/2 [==============================] - 13s 6s/step - loss: 0.1222 - val_loss: 0.1357
Epoch 6/25
2/2 [==============================] - 13s 6s/step - loss: 0.1190 - val_loss: 0.1344
Epoch 7/25
2/2 [==============================] - 15s 8s/step - loss: 0.1177 - val_loss: 0.1330
Epoch 8/25
2/2 [==============================] - 13s 6s/step - loss: 0.1163 - val_loss: 0.1331
Epoch 9/25
2/2 [==============================] - 12s 6s/step - lo

BiLSTM 구현

In [22]:
# from __future__ import print_fuction
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, Dropout
import numpy as np
import pandas as pd
from konlpy.tag import Okt, Mecab
mecab = Mecab()

stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

batch_size = 16
epochs = 25
latent_dim = 256
num_samples = 3
# max_sentence_len = 10


input_texts = []
target_texts = []


input_characters = set()
target_characters = set()
for idx in df.index :

    input_text = [ val for (val, nn) in mecab.pos(df.loc[idx, 'question'][::-1]) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]

    ans_list = [ val for (val, nn) in mecab.pos(df.loc[idx, 'answer']) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]
    ans_list.insert(0, '\t')
    ans_list.insert(-1, '\n')
    target_text = ans_list
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, dropout=0.5, return_sequences=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(latent_dim, dropout=0.5, return_sequences=True, return_state=True))(encoder(encoder_inputs))

# We discard `encoder_outputs` and only keep the states.
# state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
# state_c = Concatenate()([forward_c, backward_c]) # 셀 상태
state_h = sum([forward_h / 2, backward_h / 2]) # 은닉 상태
state_c = sum([forward_c / 2, backward_c / 2]) # 셀 상태

encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.2)

# Save modelmodel.save('s2s.h5') 
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states 

# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length) :
                stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

for seq_index in range(3):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)



Number of samples: 30
Number of unique input tokens: 1338
Number of unique output tokens: 194
Max sequence length for inputs: 695
Max sequence length for outputs: 26
Epoch 1/25
2/2 [==============================] - 9s 2s/step - loss: 0.1298 - val_loss: 0.1445
Epoch 2/25
2/2 [==============================] - 1s 262ms/step - loss: 0.1276 - val_loss: 0.1370
Epoch 3/25
2/2 [==============================] - 0s 263ms/step - loss: 0.1220 - val_loss: 0.1324
Epoch 4/25
2/2 [==============================] - 0s 266ms/step - loss: 0.1186 - val_loss: 0.1323
Epoch 5/25
2/2 [==============================] - 0s 255ms/step - loss: 0.1175 - val_loss: 0.1298
Epoch 6/25
2/2 [==============================] - 0s 266ms/step - loss: 0.1155 - val_loss: 0.1310
Epoch 7/25
2/2 [==============================] - 0s 271ms/step - loss: 0.1157 - val_loss: 0.1287
Epoch 8/25
2/2 [==============================] - 0s 266ms/step - loss: 0.1150 - val_loss: 0.1302
Epoch 9/25
2/2 [==============================] - 0s 

In [17]:
print(encoder_outputs.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, None, 512) (None, 256) (None, 256) (None, 256) (None, 256)


In [19]:
state_h.shape, state_c.shape

(TensorShape([None, 512]), TensorShape([None, 512]))

BiGRU

In [7]:
# from __future__ import print_fuction
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate, Dropout, GRU
import numpy as np
import pandas as pd
from konlpy.tag import Okt, Mecab
mecab = Mecab()

stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

batch_size = 16
epochs = 25
latent_dim = 256
num_samples = 3
# max_sentence_len = 10


input_texts = []
target_texts = []


input_characters = set()
target_characters = set()
for idx in df.index :

    input_text = [ val for (val, nn) in mecab.pos(df.loc[idx, 'question'][::-1]) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]

    ans_list = [ val for (val, nn) in mecab.pos(df.loc[idx, 'answer']) if (val, nn) not in stopword and nn in ('NNG', 'NNP', 'VV', 'VA')]
    ans_list.insert(0, '\t')
    ans_list.insert(-1, '\n')
    target_text = ans_list
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        input_characters.add(char)
    for char in target_text:
        target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(GRU(latent_dim, dropout=0.5, return_sequences=True))
encoder_outputs, state_h, state_c = Bidirectional \
  (GRU(latent_dim, dropout=0.5, return_sequences=True, return_state=True))(encoder(encoder_inputs))

# We discard `encoder_outputs` and only keep the states.
# state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
# state_c = Concatenate()([forward_c, backward_c]) # 셀 상태


encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = GRU(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = batch_size,
          epochs = epochs,
          validation_split = 0.2)

# Save modelmodel.save('s2s.h5') 
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states 

# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length) :
                stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

for seq_index in range(3):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence: ', decoded_sentence)



Number of samples: 30
Number of unique input tokens: 1338
Number of unique output tokens: 194
Max sequence length for inputs: 695
Max sequence length for outputs: 26


ValueError: ignored

In [11]:
state_h.shape

TensorShape([None, 256])


문제점 1. 한국어 -> 한국어 정확도 문제.
문제점 2. 어떤식 토근 입력?